In [1]:
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def cast_shadow(image):
    # randomly select shadow side
    shadow_side = np.random.choice(['left','right','top', 'bottom'])

    if (shadow_side == 'top') | (shadow_side == 'bottom'):
        divider_left, divider_right = np.random.randint(0, image.shape[0], 2)
    if (shadow_side == 'left') | (shadow_side == 'right'):
        divider_top, divider_bot = np.random.randint(0, image.shape[1], 2)
        
    if shadow_side == 'top':
        ul = [0, 0]
        ur = [image.shape[1], 0]
        ll = [0, divider_left]
        lr = [image.shape[1], divider_right]
    elif shadow_side == 'bottom':
        ul = [0, divider_left]
        ur = [image.shape[1], divider_right]
        ll = [0, image.shape[0]]
        lr = [image.shape[1], image.shape[0]]
    elif shadow_side == 'left':
        ul = [0, 0]
        ur = [divider_top, 0]
        ll = [0, image.shape[0]]
        lr = [divider_bot, image.shape[0]]
    elif shadow_side == 'right':
        ul = [divider_top, 0]
        ur = [image.shape[1], 0]
        ll = [divider_bot, image.shape[0]]
        lr = [image.shape[1], image.shape[0]]

    # form shadow vertices
    vertices = [np.array([ll,ul,ur,lr])]

    # create a blank mask
    mask = np.zeros_like(image[:,:,0])   

    #filling pixels inside the polygon defined by "vertices" with the fill color    
    mask = cv2.fillPoly(mask, vertices, 255).astype('bool')

    # shadow-masked area will be halved its luminanse
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    hls[:,:,1][mask]=hls[:,:,1][mask] * np.random.uniform(0.1, 0.3, 1)

    return cv2.cvtColor(hls.astype('uint8'),cv2.COLOR_HLS2BGR)

In [3]:
lines = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []
for line in lines:
    for i in range(1):
        source_path = line[i]
        image = cv2.imread(source_path)
        image = cast_shadow(image)
        #image_flipped = np.fliplr(image)
        #image = random_brightness(image)
        images.append(image)
        #images.append(image_flipped)
        measurement = float(line[3])
        #measurement_flipped = -measurement
        measurements.append(measurement)
        #measurements.append(measurement_flipped)

        
X_train = np.array(images)
y_train = np.array(measurements)
print(X_train.shape)
print(y_train.shape)

(4141, 160, 320, 3)
(4141,)


In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Lambda, Activation, Cropping2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))
model.add(Conv2D(24, (5, 5), activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(36, (5, 5), activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(48, (5, 5), activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), activation="relu", data_format='channels_first'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(60))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer= 'adam')
model.fit(X_train, y_train, validation_split = 0.2, shuffle=True, epochs=5)

model.save('model2.h5')

Using TensorFlow backend.
D:\Anaconda\envs\Tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\Tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\Tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\Tensorflow\lib\site-packages\tensorflow\python\framewo

NameError: name 'X_train' is not defined